# Beurteilung der Datenqualität

Daten sind die Basis für jedes Machine Learning-Projekt. Daher ist die Beurteilung der Datenqualität am Anfang extrem wichtig. Nur so kannst du sicherstellen, dass die Ergebnisse auch wirklich belastbar sind.

Daten können nach unterschiedlichen Kriterien bewertet werden. Häufig wirst du dazu statistische Methoden nutzen, Korrelationen berechen oder einfach fehlende oder nicht aktuelle Daten erkennen.

## Statistik

In diesem Fall arbeitest du mit existierenden Daten, um die Methoden genauer kennenzulernen. Dazu nutzt du die Daten von Eurostat, der europäischen Statistikbehörde:

In [ ]:
!pip install eurostat

Alle Statistikdaten haben bei Eurostat ein Kürzel. In diesem Beispiel beschäftigst du dich mit den *Hauspreisen*, diese tragen das Kürzel `prc_hpi_a`.  Eine genauere Erklärung findest du [hier](https://ec.europa.eu/eurostat/cache/metadata/en/prc_hpi_inx_esms.htm). 

Das Herunterladen der Daten ist nun sehr einfach:

In [ ]:
import eurostat
df = eurostat.get_data_df("prc_hpi_a")
df

Darin sind jede Menge Einzeldaten enthalten. Du interessierst dich für den Index der Hauspreise, bei dem als Grundlage da Jahr 2010 verwendet wurde und dort für die neuen Häuser:

In [ ]:
hp10 = df[(df["unit"] == "I10_A_AVG") & (df["purchase"] == "DW_NEW")].copy()
hp10

Du kannst nun eine Zusammenfassung der Werte aufrufen:

In [ ]:
hp10.describe()

Hier siehst du zuerst, dass die Werte auf das Jahr 2010 normiert wurden!

Wie du siehst, werden in den unterschiedlichen Spalten nicht immer gleich viel Werte angezeigt. Das liegt daran, dass nicht vorhandene Werte (`NaN` für *not a number*) dort nicht mitgezählt werden. 2005 fehlen also noch viele Werte! Das muss du unbedingt beachten, wenn du die Daten analysierst.

Möchtest du nur vollständige Datensätze betrachten, kannst du `.dropna()` verwenden:

In [ ]:
hp10.dropna()

Wie du siehst, hat sich die Datenmenge dadurch enorm reduziert. Das kann eine sinnvolle Einschränkung sein, muss aber nicht immer die richtige Lösung sein. Du kannst das `.dropna()` auch nur für bestimmte Spalten verwenden. Wenn du also nur vollständige Daten möchtest, die sich auf die Jahre 2010-2021 beziehen:

In [ ]:
hp10.columns

In [ ]:
columns = [str(y) for y in range(2010,2023)]
hp10.dropna(subset=columns)

Das sind jetzt wieder viel mehr Länder. Sinnvollerweise kannst du damit eine Analyse durchführen, die Daten genauer dieser Jahre enthält:

In [ ]:
hp_from_2010 = hp10.dropna(subset=columns)[["geo\\TIME_PERIOD"] + columns]
hp_from_2010

### Datenstruktur anpassen

Für die weitere Analyse ist es sinnvoll, die Datenstruktur etwas anzupassen. So ist `geo\time` eine etwas unglückliche Bezeichnung, hier ist nur das Land gemeint, das kannst du einfach umbenennen:

In [ ]:
hp_from_2010.rename(columns={"geo\\TIME_PERIOD": "country"}, inplace=True)
hp10.rename(columns={"geo\\TIME_PERIOD": "country"}, inplace=True)

### Zeitreihen

Nun kannst du dir die Immobilienpreisentwicklung in Deutschland anschauen:

In [ ]:
hp10[hp10["country"] == "DE"]

`94` ist dabei die ursprüngliche Zeile im `DataFrame`.

Für die Visualisierung betrachtest du nur die Jahre als Spalten und transponierst das Ergebnis:

In [ ]:
hp10[hp10["country"] == "DE"][columns].T.plot.bar()

### Zeitreihen vieler Daten

Dafür betrachtest du die saisonjustierten Daten zu einem bestimmten Zeitpunkt, allerdings für alle Länder gleichzeitig. Die `pivot`-Funktion kennst du schon, die ist hier sehr nützlich:

In [ ]:
country_prices = hp10[["country"]+[str(y) for y in range(2005,2022)]].set_index("country")
country_prices.T.plot(figsize=(16,9))

Das ist etwas unübersichtlich und die fehlenden Daten kannst du gar nicht gut erkennen. Auch ist es schwierig, das Land mit dem "Boom" eindeutig zu identifizieren.

Manchmal ist eine *qualitative Darstellung* dafür besser geeignet.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
sns.heatmap(country_prices)

Das ist viel besser, um Fehler oder Ausreißer zu erkennen. Offenbar ist in der Türkei der Markt explodiert, während in UK keine Daten mehr für 2020 bereitgestellt werden. Auch die ersten Datenpunkte für die Länder kannst du auf einen Blick erkennen!

### Korrelation der Daten

Nun betrachtest du die Hauspreise zueinander. Oftmals wirst du es mit Größen zu tun haben, die gar nicht unabhängig voneinander sind. Um das zu ermitteln, gibt es in Python und `pandas` leistungsfähige Funktionen.

Bestimmt hängen die Hauspreise in Deutschland und Österreich eng zusammen. das kannst du grafisch über einen sog. *Pairplot* ermitteln:

In [ ]:
sns.jointplot(x=country_prices.T["DE"], y=country_prices.T["AT"], scatter=False, kind="reg")

Den Zusammenhang kannst du auf den ersten Blick erkennen! In Österreich war die Steigerungsrate etwas größer.

Mit anderen Ländern wird das sicher anders aussehen, schau dir zum Vergleich noch die Türkei an:

In [ ]:
sns.jointplot(x=country_prices.T["DE"], y=country_prices.T["TR"], scatter=False, kind="reg")

Auch diese Preise sind sehr stark voneinander abhängig. In der Türkei gab es eine massive Preissteigerung, die aber proportional zur deutschen ist.

Mithilfe des sog. Pearson-Koeffizienten kannst du ermitteln, wie stark die Daten korreliert sind:

In [ ]:
country_prices.T[["DE", "AT", "TR"]].corr()

Die Korrelation ist also mit Österreich nur minimal höher (1 steht für vollkommen korreliert, -1 für vollkommen antikorreliert, 0 für unkorreliert).

Mit solchen Daten muss du also vorsichtig sein, weil du nicht davon ausgehen kannst, dass diese unabhängig voneinander sind. Damit kannst du leicht in das sog. *Overfitting* kommen.

Wenn du dir einen grafischen Überblick über diese Länder verschaffen willst, kannst du dazu einen sog. *Pairplot* einsetzen:

In [ ]:
sns.pairplot(country_prices.T[["DE", "AT", "TR"]])

Hier siehst du noch etwas genauer die Zusammenhänge zwischen den Immobilienpreisen in den jeweiligen Ländern. Der lineare Zusammenhang ist ganz offensichtlich.

## Boxplots

Wie du in den oberen Grafiken erkennen kannst, variiert die Schwankungsbreite der Preise erheblich von Land zu Land. Auch diese kannst du dir grafisch anzeigen lassen, dazu dienen sog. *Boxplots*.

In [ ]:
import pandas as pd
u = country_prices.melt(ignore_index=False, var_name="year", value_name="price_index")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
sns.boxplot(y = u.index, x = u["price_index"])

Hier kannst du viele interessante Details entdecken. So ist die Preisspanne in der Türkei bei weitem am höchsten. Auf der anderen Seite gibt es auch Länder, in denen zumindest einmal auch die Preise zurückgegangen sind (Niveau unter 100). In Irland ist das besonders ausgeprägt - bestimmt ein Effekt der Finanzkrise.

## Datenqualität und Statistik kann spannend sein

Auch wenn es etwas *trocken* klingt, kannst du die Untersuchung der Datenqualität mithilfe von aussagekräftigen Diagrammen richtig interessant gestalten. Oft wirst du in diesem Stadium schon interessante Einblicke haben, die es dir viel leichter machen, dich mit Machine Learning und Advanced Analytics zu beschäftigen.

Ab und zu hingegen kann es auch passieren, dass du die Daten als unbrauchbar erkennst. Und in vielen Fällen kannst du die Zwischenergebnisse bereits präsentieren und damit Aufmerksamkeit erregen.